In [72]:
'''
  code by Minho Ryu @bzantium
  
'''
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense
import numpy as np

tf.reset_default_graph()

sentence = (
    'Lorem ipsum dolor sit amet consectetur adipisicing elit '
    'sed do eiusmod tempor incididunt ut labore et dolore magna '
    'aliqua Ut enim ad minim veniam quis nostrud exercitation'
)

word_dict = {w: i for i, w in enumerate(list(set(sentence.split())))}
word_dict['<E>'] = len(word_dict)
number_dict = {i: w for w, i in word_dict.items()}
vocab_size = len(word_dict)
n_embed = 5
n_class = len(word_dict)
n_step = len(sentence.split())
n_hidden = 5
n_batch = 32

def make_batch(sentence, n_batch):
    words = sentence.split()
    input_batch = []
    target_batch = []
    for _ in range(n_batch):
        input_batch.append([word_dict[n] for n in words])
        target_batch.append([word_dict[n] for n in (sentence + ' <E>').split()[1:]])

    return input_batch, target_batch

# Bi-LSTM Model
class biLSTM(object):
    def __init__(self, sess, vocab_size, n_embed, n_step, n_hidden, n_class):
        self.sess = sess
        self.X = tf.placeholder(tf.int32, [None, n_step])
        self.Y = tf.placeholder(tf.int32, [None, n_step])
        embedding = Embedding(vocab_size, n_embed)
        inputs = embedding(self.X)
        
        fc = Dense(n_class, input_shape=(n_hidden * 2,))

        lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
        lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)

        # outputs : [batch_size, len_seq, n_hidden], states : [batch_size, n_hidden]
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, inputs, dtype=tf.float32)

        outputs = tf.concat([outputs[0], outputs[1]], -1) # output[0] : lstm_fw, output[1] : lstm_bw 
        logits = fc(outputs) # [batch_size, n_step, 2 * n_hidden] -> [batch_size, n_step, n_class]

        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(0.001).minimize(self.cost)

        self.prediction = tf.cast(tf.argmax(logits, -1), tf.int32)

        tf.global_variables_initializer().run()
    
    def train(self, inputs, labels):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: inputs, self.Y: labels})
     
    def predict(self, inputs):
        return self.sess.run(self.prediction, feed_dict={self.X: inputs})

# Training
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
with tf.Session(config=run_config) as sess:
    input_batch, target_batch = make_batch(sentence, n_batch)
    model = biLSTM(sess, vocab_size, n_embed, n_step, n_hidden, n_class)
    for epoch in range(1000):
        loss, _ = model.train(input_batch, target_batch)
        if (epoch + 1)%100 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    predict =  model.predict([input_batch[0]])
    
print(sentence)
print([number_dict[n] for n in predict[0]])

Epoch: 0100 cost = 3.004276
Epoch: 0200 cost = 2.434045
Epoch: 0300 cost = 2.067851
Epoch: 0400 cost = 1.775385
Epoch: 0500 cost = 1.505942
Epoch: 0600 cost = 1.273642
Epoch: 0700 cost = 1.060639
Epoch: 0800 cost = 0.855925
Epoch: 0900 cost = 0.668118
Epoch: 1000 cost = 0.521641
Lorem ipsum dolor sit amet consectetur adipisicing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua Ut enim ad minim veniam quis nostrud exercitation
['ipsum', 'dolor', 'sit', 'amet', 'consectetur', 'adipisicing', 'elit', 'sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua', 'Ut', 'enim', 'ad', 'minim', 'veniam', 'quis', 'nostrud', 'exercitation', '<E>']
